Mandatory packages to be imported

In [ ]:
!pip install numpy
!pip install yfinance
!pip install pandas

In [4]:
import numpy as np
import pandas as pd
import yfinance as yf

Take in VIX data (1990 till date)

In [10]:
vix_df = yf.download('^VIX')

[*********************100%***********************]  1 of 1 completed


In [12]:
vix_df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1990-01-02,17.240000,17.240000,17.240000,17.240000,17.240000,0
1990-01-03,18.190001,18.190001,18.190001,18.190001,18.190001,0
1990-01-04,19.219999,19.219999,19.219999,19.219999,19.219999,0
1990-01-05,20.110001,20.110001,20.110001,20.110001,20.110001,0
1990-01-08,20.260000,20.260000,20.260000,20.260000,20.260000,0
...,...,...,...,...,...,...
2022-05-23,28.980000,30.389999,28.290001,28.480000,28.480000,0
2022-05-24,29.430000,31.070000,29.040001,29.450001,29.450001,0
2022-05-25,29.330000,30.230000,28.160000,28.370001,28.370001,0


Strategy specification (1-month Moving Average)